In [ ]:
!pip install fuzzywuzzy
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzywuzzy.fuzz import token_set_ratio
from fuzzywuzzy.fuzz import token_sort_ratio

In [ ]:
df = pd.read_csv('output.csv')

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_n_words,q2_n_words,word_common,word_total,q1len,q2len,word_share,freq_qid1,freq_qid2,freq_q1+freq_q2,freq_q1-freq_q2
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,14.0,12.0,11.0,12.0,65.0,56.0,0.916667,1.0,1.0,2.0,0.0
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0,8.0,13.0,4.0,14.0,46.0,83.0,0.285714,4.0,2.0,6.0,2.0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,14.0,10.0,4.0,20.0,72.0,58.0,0.200000,1.0,1.0,2.0,0.0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0,11.0,9.0,0.0,19.0,48.0,55.0,0.000000,1.0,1.0,2.0,0.0
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0,13.0,7.0,4.0,16.0,73.0,38.0,0.250000,3.0,1.0,4.0,2.0


In [ ]:
df.shape

(404351, 17)

# **Advance Fearturization**
Terms:
1. **Token**: Each word seperated in a sentence.
2. **stop_word**: words imported frm NLTK.
3. **Word**: Every token which is not a stopword.

Features:

* **cwc_min** = common_word_count / min(len(Q1_words),len(Q2_words))

* **cwc_max** = common_word_count / max(len(Q1_words),len(Q2_words))

* **csc_min** = common_stop_count / min(len(Q1_words),len(Q2_words))

* **csc_max** = common_stop_count / max(len(Q1_words),len(Q2_words))

* **ctc_min** = common_token_count / min(len(Q1_words),len(Q2_words))

* **ctc_max** = common_token_count / max(len(Q1_words),len(Q2_words))

* **last_word_eq** = Check if last word of both question Q1 and Q2 are equal or not last_word_eq = int(q1_tokens[-1]== q2_tokens[-1])

* **first_word_eq** = Check if first word of both question Q1 and Q2 are equal or not first_word_eq = int(q1_tokens[0]== q2_tokens[0])

* **abs_len_diff** = abs(len(q1_tokens) - len(q2_tokens))

* **mean_len** = (len(q1_tokens) + len(q2_tokens))/2


 pip install fuzzywuzzy

* **fuzzy_ratio**

* **fuzzy_partial_ratio**

* **token_sort_ratio**

* **token_set_ratio**

* **longest_substr_ratio** = Ratio of length longest common substring to min length of token countof q1 and 2
longest_substr_ratio = len(longest common string) / min(len(q1_tokens), len(q2_tokens))

In [ ]:

def pre_process(x):
  x = str(x)
  x = x.replace(",000,000","m").replace(",000","k").replace(",","'").replace("%","percent")
  x = re.sub(r"([0-9]+)000000",r"\\lm",x)
  x = re.sub(r"([0-9]+)000",r"\\lk",x)

  porter = PorterStemmer()
  pattern = re.compile('\W')

  if type(x) == type(''):
    x = re.sub(pattern, ' ', x)

  if type(x) == type(''):
    x = porter.stem(x)
    example1 = BeautifulSoup(x)
    x = example1.get_text()

  return x


In [ ]:
df['question1'] = df['question1'].apply(pre_process)
df['question2'] = df['question2'].apply(pre_process)

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_n_words,q2_n_words,word_common,word_total,q1len,q2len,word_share,freq_qid1,freq_qid2,freq_q1+freq_q2,freq_q1-freq_q2
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,14.0,12.0,11.0,12.0,65.0,56.0,0.916667,1.0,1.0,2.0,0.0
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0,8.0,13.0,4.0,14.0,46.0,83.0,0.285714,4.0,2.0,6.0,2.0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,14.0,10.0,4.0,20.0,72.0,58.0,0.200000,1.0,1.0,2.0,0.0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0,11.0,9.0,0.0,19.0,48.0,55.0,0.000000,1.0,1.0,2.0,0.0
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt wat,0,13.0,7.0,4.0,16.0,73.0,38.0,0.250000,3.0,1.0,4.0,2.0


In [ ]:
for i in range(len(df['question1'])):
  df.loc[i, 'fuzzy_ratio'] = fuzz.ratio(df['question1'][i], df['question2'][i])
  df.loc[i, 'fuzzy_partial_ratio'] = fuzz.partial_ratio(df['question1'][i], df['question2'][i])
  df.loc[i, 'token_sort_ratio'] = token_sort_ratio(df['question1'][i], df['question2'][i])
  df.loc[i, 'token_set_ratio'] = token_set_ratio(df['question1'][i], df['question2'][i])

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_n_words,q2_n_words,word_common,word_total,...,q2len,word_share,freq_qid1,freq_qid2,freq_q1+freq_q2,freq_q1-freq_q2,fuzzy_ratio,fuzzy_partial_ratio,token_sort_ratio,token_set_ratio
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,14.0,12.0,11.0,12.0,...,56.0,0.916667,1.0,1.0,2.0,0.0,93.0,100.0,93.0,100.0
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0,8.0,13.0,4.0,14.0,...,83.0,0.285714,4.0,2.0,6.0,2.0,64.0,72.0,62.0,81.0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,14.0,10.0,4.0,20.0,...,58.0,0.200000,1.0,1.0,2.0,0.0,43.0,46.0,64.0,65.0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0,11.0,9.0,0.0,19.0,...,55.0,0.000000,1.0,1.0,2.0,0.0,8.0,10.0,25.0,26.0
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt wat,0,13.0,7.0,4.0,16.0,...,38.0,0.250000,3.0,1.0,4.0,2.0,39.0,53.0,44.0,53.0


In [ ]:
stopwords = set(stopwords.words('english'))

token_words_1 = []
stop_words_1 = []
word_1 = []

token_words_2 = []
stop_words_2 = []
word_2 = []

for i in range(len(df['question1'])):
  w1 = []
  w2 = []
  w3 = []
  w4 = []

  q1 = df['question1'][i].split(' ')
  q2 = df['question2'][i].split(' ')

  token_words_1.append(q1)
  token_words_2.append(q2)

  for w in q1:
    if w in stopwords:
      w1.append(w)
    else:
      w2.append(w)
  for w in q2:
    if w in stopwords:
      w3.append(w)
    else:
      w4.append(w)

  stop_words_1.append(w1)
  stop_words_2.append(w3)

  word_1.append(w2)
  word_2.append(w4)

In [ ]:
df.to_csv('fuzzy.csv', index=False)

In [ ]:
df = pd.read_csv('fuzzy.csv')

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_n_words,q2_n_words,word_common,word_total,...,q2len,word_share,freq_qid1,freq_qid2,freq_q1+freq_q2,freq_q1-freq_q2,fuzzy_ratio,fuzzy_partial_ratio,token_sort_ratio,token_set_ratio
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,14.0,12.0,11.0,12.0,...,56.0,0.916667,1.0,1.0,2.0,0.0,93.0,100.0,93.0,100.0
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0,8.0,13.0,4.0,14.0,...,83.0,0.285714,4.0,2.0,6.0,2.0,64.0,72.0,62.0,81.0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,14.0,10.0,4.0,20.0,...,58.0,0.200000,1.0,1.0,2.0,0.0,43.0,46.0,64.0,65.0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0,11.0,9.0,0.0,19.0,...,55.0,0.000000,1.0,1.0,2.0,0.0,8.0,10.0,25.0,26.0
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt wat,0,13.0,7.0,4.0,16.0,...,38.0,0.250000,3.0,1.0,4.0,2.0,39.0,53.0,44.0,53.0


In [ ]:
print(token_words_1[1])
len(token_words_1[1])

['what', 'is', 'the', 'story', 'of', 'kohinoor', 'kohinoor', 'diamond']


8

In [ ]:
for i in range(len(df['question1'])):
  df.loc[i, 'common_word_count'] = len(set(word_1[i]).intersection(set(word_2[i])))
  df.loc[i, 'common_stop_count'] = len(set(stop_words_1[i]).intersection(set(stop_words_2[i])))
  df.loc[i, 'common_token_count'] = len(set(token_words_1[i]).intersection(set(token_words_2[i])))

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_n_words,q2_n_words,word_common,word_total,...,freq_qid2,freq_q1+freq_q2,freq_q1-freq_q2,fuzzy_ratio,fuzzy_partial_ratio,token_sort_ratio,token_set_ratio,common_word_count,common_stop_count,common_token_count
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,14.0,12.0,11.0,12.0,...,1.0,2.0,0.0,93.0,100.0,93.0,100.0,5.0,6.0,11.0
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0,8.0,13.0,4.0,14.0,...,2.0,6.0,2.0,64.0,72.0,62.0,81.0,2.0,2.0,4.0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,14.0,10.0,4.0,20.0,...,1.0,2.0,0.0,43.0,46.0,64.0,65.0,2.0,2.0,4.0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0,11.0,9.0,0.0,19.0,...,1.0,2.0,0.0,8.0,10.0,25.0,26.0,0.0,0.0,0.0
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt wat,0,13.0,7.0,4.0,16.0,...,1.0,4.0,2.0,39.0,53.0,44.0,53.0,1.0,2.0,3.0


In [ ]:
for i in range(len(df['question1'])):
  df.loc[i, 'cwc_min'] = df['common_word_count'][i] / min(len(word_1[i]), len(word_2[i]))
  df.loc[i, 'cwc_max'] = df['common_word_count'][i] / max(len(word_1[i]), len(word_2[i]))
  df.loc[i, 'csc_min'] = df['common_stop_count'][i] / min(len(stop_words_1[i]), len(stop_words_2[i]))
  df.loc[i, 'csc_max'] = df['common_stop_count'][i] / max(len(stop_words_1[i]), len(stop_words_2[i]))
  df.loc[i, 'ctc_min'] = df['common_token_count'][i] / min(len(token_words_1[i]), len(token_words_2[i]))
  df.loc[i, 'ctc_max'] = df['common_token_count'][i] / max(len(token_words_1[i]), len(token_words_2[i]))

<ipython-input-52-10fdb91aa80f>:4: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[i, 'csc_min'] = df['common_stop_count'][i] / min(len(stop_words_1[i]), len(stop_words_2[i]))
<ipython-input-52-10fdb91aa80f>:4: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[i, 'csc_min'] = df['common_stop_count'][i] / min(len(stop_words_1[i]), len(stop_words_2[i]))
<ipython-input-52-10fdb91aa80f>:4: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[i, 'csc_min'] = df['common_stop_count'][i] / min(len(stop_words_1[i]), len(stop_words_2[i]))
<ipython-input-52-10fdb91aa80f>:4: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[i, 'csc_min'] = df['common_stop_count'][i] / min(len(stop_words_1[i]), len(stop_words_2[i]))
<ipython-input-52-10fdb91aa80f>:4: RuntimeWarning: invalid value encountered in scalar divide
  df.loc[i, 'csc_min'] = df['common_stop_count'][i] / min(len(stop_words_1[i]), len(stop_words_2[i]))
<ipython-input-52-10

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_n_words,q2_n_words,word_common,word_total,...,common_stop_count,common_token_count,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,first_word_eq,last_word_eq
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,14.0,12.0,11.0,12.0,...,6.0,11.0,0.833333,0.714286,1.0,0.857143,0.916667,0.785714,1.0,0.0
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0,8.0,13.0,4.0,14.0,...,2.0,4.0,0.500000,0.222222,0.5,0.500000,0.500000,0.307692,1.0,0.0
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,14.0,10.0,4.0,20.0,...,2.0,4.0,0.400000,0.333333,0.4,0.250000,0.400000,0.285714,1.0,0.0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0,11.0,9.0,0.0,19.0,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt wat,0,13.0,7.0,4.0,16.0,...,2.0,3.0,0.200000,0.100000,1.0,0.666667,0.428571,0.230769,1.0,0.0


In [ ]:
nan_count = df['csc_min'].isna().sum()

if nan_count > 0:
  print(f"Number of NaN values in 'csc_min': {nan_count}")
  df = df.dropna(subset=['csc_min'])
  print(f"NaN values in 'csc_min' removed. New DataFrame shape: {df.shape}")
else:
  print("No NaN values found in 'csc_min'")


1681

In [ ]:
token_words_1[1]
token_words_1[1][1]

'is'

In [ ]:
for i in range(len(df['question1'])):
    q1_tokens = token_words_1[i][0]
    q2_tokens = token_words_2[i][0]

    if len(q1_tokens) == len(q2_tokens):
        df.loc[i, 'first_word_eq'] = 1
    else:
        df.loc[i, 'first_word_eq'] = 0
for i in range(len(df['question1'])):
    q1_tokens = token_words_1[i][-1]
    q2_tokens = token_words_2[i][-1]

    if len(q1_tokens) == len(q2_tokens):
        df.loc[i, 'last_word_eq'] = 1
    else:
        df.loc[i, 'last_word_eq'] = 0

In [ ]:
for i in range(len(df['question1'])):
  x = len(token_words_1[i])
  y = len(token_words_2[i])
  df.loc[i, 'abs_len_diff'] = abs(x - y)
  df.loc[i, 'mean_len'] = (x+y)/2

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_n_words,q2_n_words,word_common,word_total,...,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,first_word_eq,last_word_eq,abs_len_diff,mean_len
0,0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,14.0,12.0,11.0,12.0,...,0.833333,0.714286,1.0,0.857143,0.916667,0.785714,1.0,0.0,2.0,13.0
1,1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0,8.0,13.0,4.0,14.0,...,0.500000,0.222222,0.5,0.500000,0.500000,0.307692,1.0,0.0,5.0,10.5
2,2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0,14.0,10.0,4.0,20.0,...,0.400000,0.333333,0.4,0.250000,0.400000,0.285714,1.0,0.0,4.0,12.0
3,3,7,8,why am i mentally very lonely how can i solve it,find the remainder when math2324math is divide...,0,11.0,9.0,0.0,19.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,2.0,10.0
4,4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt wat,0,13.0,7.0,4.0,16.0,...,0.200000,0.100000,1.0,0.666667,0.428571,0.230769,1.0,0.0,6.0,10.0
